# Upload records on the OSSR


To upload records on the OSSR, you may use the [Zenodo client](https://gitlab.com/escape-ossr/eossr/-/blob/master/eossr/api/zenodo.py).

The client is also used by the continuous integration to automatise project's uploads from GitLab to Zenodo [check the EOSSR scripts here](https://gitlab.com/escape-ossr/eossr/-/tree/master/eossr/scripts).

**However**, to upload records to the OSSR, you **will need** to use your Zenodo token ([create one](https://zenodo.org/account/settings/applications/tokens/new/)).


In [ ]:
import os

token = os.getenv("SANDBOX_ZENODO_TOKEN")  # Replace with your own token

if not token:
    raise SystemExit("You need to use a token for that notebook!")

In [ ]:
import json
from eossr.api.zenodo import ZenodoAPI, Record

# Please note that for this demo we are using Zenodo Sandbox.
zen = ZenodoAPI(access_token=token, sandbox=True)

In [ ]:
# Test token
user_deposits_response = zen.query_user_deposits()

## Create a new deposit

First we would need to create a new deposit


In [ ]:
new_deposit_response = zen.create_new_deposit()

### Zenodo HTTP status

The function `create_new_deposit` would have raised an error in case of failure.
But if you can its status code meaning using our `ZenodoHTTPStatus` class


In [ ]:
from eossr.api.zenodo.http_status import ZenodoHTTPStatus

In [ ]:
status_new_deposit = ZenodoHTTPStatus(new_deposit_response)
print(status_new_deposit)

## Add files to the deposit

Then we upload to the new deposit a file (let's say a `codemeta.json` metadata file)


In [ ]:
deposit_id = new_deposit_response.json()["id"]
upload_file_response = zen.upload_file_deposit(
    deposit_id, name_file="codemeta.json", path_file="../../codemeta.json"
)

status_upload = ZenodoHTTPStatus(upload_file_response)
print(status_upload)

## Add metadata to the deposit

We add metadata to the Zenodo deposit using the EOSSR codemeta2zenodo conversor


In [ ]:
from eossr.metadata.codemeta2zenodo import parse_codemeta_and_write_zenodo_metadata_file

In [ ]:
parse_codemeta_and_write_zenodo_metadata_file(codemeta_filename='../../codemeta.json',
                                              outdir='.',
                                              add_escape2020=True,
                                              overwrite=True,
                                              )

In [ ]:
# Let's have a look to the output to the .zenodo.json file
! cat .zenodo.json

In [ ]:
with open('.zenodo.json') as f:
    zenodo_metadata = json.load(f)

update_metadata_info_response = zen.set_deposit_metadata(
    deposit_id, zenodo_metadata)
status_metadata = ZenodoHTTPStatus(update_metadata_info_response)
print(status_metadata)

### Check the content of the new deposit

The `request` answers contain all the information of the current upload.  
One can check the metadata in the JSON dict:


In [ ]:
update_metadata_info_response.json()['metadata']

---
We can check the files that we have already uploaded


In [ ]:
update_metadata_info_response.json()["files"]

## Publish the deposit - or not.

Finally we can publish the new deposit.
However in this case, because this is just a demo, we will just erase the deposit.


In [ ]:
# In case you would like to publish the deposit, just uncomment the following lines.

# # publish_response = z.publish_deposit(deposit_id)
# # status = ZenodoHTTPStatus(publish_response)

In [ ]:
from eossr.api.zenodo.http_status import HTTPStatusError

In [ ]:
erase_deposit = zen.erase_deposit(deposit_id)